In [1]:
import torch
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import os
import torch
from torch import nn
from torch import nn, optim

import torch.nn.functional as F

%matplotlib inline

In [2]:
# loans =  pd.read_csv("/s/chopin/d/proj/ramfis-aida/MachineTranslationIPA/datasets/Hindi-Persian-Loans.csv")
# equiv = pd.read_csv("/s/chopin/d/proj/ramfis-aida/MachineTranslationIPA/datasets/Hindi-Persian-Synonyms.csv")
# random = pd.read_csv("/s/chopin/d/proj/ramfis-aida/MachineTranslationIPA/datasets/Hindi-Persian-Randoms.csv")
# hard_neg = pd.read_csv("/s/chopin/d/proj/ramfis-aida/MachineTranslationIPA/datasets/Hindi-Persian-Hard-Negatives.csv")

import pandas as pd
import io
import requests
import csv
 
# s=requests.get(url).content
# c=pd.read_csv(io.StringIO(s.decode('utf-8')))
#equiv = pd.read_csv("/s/chopin/d/proj/ramfis-aida/MachineTranslationIPA/datasets/Hindi-Persian-Synonyms.csv")
# rand = pd.read_csv("/s/chopin/d/proj/ramfis-aida/MachineTranslationIPA/datasets/Hindi-Persian-Randoms.csv")
# train = pd.read_csv("/s/chopin/d/proj/ramfis-aida/MachineTranslationIPA/train_final.csv")
# test =pd.read_csv("/s/chopin/d/proj/ramfis-aida/MachineTranslationIPA/test_final.csv")
url_syn = 'https://raw.githubusercontent.com/csu-signal/loan-word-detection/main/Datasets/Synonyms/Hindi-Persian-Synonyms.csv'
url_rand = 'https://raw.githubusercontent.com/csu-signal/loan-word-detection/main/Datasets/Randoms/Hindi-Persian-Randoms.csv'
url_loans = 'https://raw.githubusercontent.com/csu-signal/loan-word-detection/main/Datasets/Loans/Hindi-Persian-Loans.csv'
url_hard = 'https://raw.githubusercontent.com/csu-signal/loan-word-detection/main/Datasets/Hard-Negatives/Hindi-Persian-Hard-Negatives.csv'

url_syn = requests.get(url_syn).content
url_rand = requests.get(url_rand).content
url_loans = requests.get(url_loans).content
url_hard = requests.get(url_hard).content
#c = pd.read_csv(s)
#df_1 = pd.read_csv(url, header=None, sep='\n')
equiv=pd.read_csv(io.StringIO(url_syn.decode('utf-8')))
random = pd.read_csv(io.StringIO(url_rand.decode('utf-8')))
loans=pd.read_csv(io.StringIO(url_loans.decode('utf-8')))
hard_neg = pd.read_csv(io.StringIO(url_hard.decode('utf-8')))





In [3]:
equiv.shape

(6207, 14)

In [4]:
loans.shape,equiv.shape,random.shape, hard_neg.shape

((1344, 14), (6207, 14), (1344, 14), (4282, 14))

In [5]:
#first shuffle all the different sets before creating the splits:
loans = loans.iloc[np.random.permutation(len(loans))]
equiv = equiv.iloc[np.random.permutation(len(equiv))]
random = random.iloc[np.random.permutation(len(random))]
hard_neg = hard_neg.iloc[np.random.permutation(len(hard_neg))]

In [6]:
loans.shape,equiv.shape,random.shape, hard_neg.shape

((1344, 14), (6207, 14), (1344, 14), (4282, 14))

In [40]:
loans_realdist_frac = .3
equiv_nonloan_frac = .4
random_nonloan_frac = .2
hard_neg_nonloan_frac = .1
nonloan_frac = equiv_nonloan_frac+random_nonloan_frac+hard_neg_nonloan_frac

In [41]:
train_frac = .9

In [42]:
# train sets for previous versions of datasets

# version one: real distribution - train sets for new versions with updated datasets
# version one with 30% loan, 40% equiv, 20% random, and 10% hard neg

loans_train_realdist = loans.iloc[:int(loans.shape[0] * train_frac), :]
loans_test_realdist = loans.iloc[int(loans.shape[0] * train_frac):, :]
equiv_train_realdist = equiv.iloc[:int(loans.shape[0] * equiv_nonloan_frac/loans_realdist_frac * train_frac), :]
equiv_test_realdist = equiv.iloc[int(loans.shape[0] * equiv_nonloan_frac/loans_realdist_frac * train_frac):int(loans.shape[0] * equiv_nonloan_frac/loans_realdist_frac), :]
random_train_realdist = random.iloc[:int(loans.shape[0] * random_nonloan_frac/loans_realdist_frac * train_frac), :]
random_test_realdist = random.iloc[int(loans.shape[0] * random_nonloan_frac/loans_realdist_frac * train_frac):int(loans.shape[0] * random_nonloan_frac/loans_realdist_frac), :]
hard_neg_train_realdist = hard_neg.iloc[:int(loans.shape[0] * hard_neg_nonloan_frac/loans_realdist_frac * train_frac), :]
hard_neg_test_realdist = hard_neg.iloc[int(loans.shape[0] * hard_neg_nonloan_frac/loans_realdist_frac * train_frac):int(loans.shape[0] * hard_neg_nonloan_frac/loans_realdist_frac), :]

#version two: balanced - train sets for new versions with updated datasets, version two with 50 percent balanced loans and rest: 1/7 hard neg, 4/7 synonyms, 2/7 random pairs

loans_train_balanced = loans.iloc[:int(loans.shape[0] * train_frac), :]
loans_test_balanced = loans.iloc[int(loans.shape[0] * train_frac):, :]
equiv_train_balanced = equiv.iloc[:int(loans.shape[0] * equiv_nonloan_frac/nonloan_frac * train_frac), :]
equiv_test_balanced = equiv.iloc[int(loans.shape[0] * equiv_nonloan_frac/nonloan_frac * train_frac):int(loans.shape[0] * equiv_nonloan_frac/nonloan_frac), :]
random_train_balanced = random.iloc[:int(loans.shape[0] * random_nonloan_frac/nonloan_frac * train_frac), :]
random_test_balanced = random.iloc[int(loans.shape[0] * random_nonloan_frac/nonloan_frac * train_frac):int(loans.shape[0] * random_nonloan_frac/nonloan_frac), :]
hard_neg_train_balanced = hard_neg.iloc[:int(loans.shape[0] * hard_neg_nonloan_frac/nonloan_frac * train_frac), :]
hard_neg_test_balanced = hard_neg.iloc[int(loans.shape[0] * hard_neg_nonloan_frac/nonloan_frac * train_frac):int(loans.shape[0] * hard_neg_nonloan_frac/nonloan_frac), :]

#version three: using the entire dataset, overweighted against loan words

loans_train_alldata = loans.iloc[:int(loans.shape[0] * train_frac), :] 
loans_test_alldata = loans.iloc[int(loans.shape[0] * train_frac):, :]
equiv_train_alldata = equiv.iloc[:int(equiv.shape[0] * train_frac), :]
equiv_test_alldata = equiv.iloc[int(equiv.shape[0] * train_frac):, :]
random_train_alldata = random.iloc[:int(random.shape[0] * train_frac), :]
random_test_alldata = random.iloc[int(random.shape[0] * train_frac):, :]
hard_neg_train_alldata = hard_neg.iloc[:int(hard_neg.shape[0] * train_frac), :]
hard_neg_test_alldata = hard_neg.iloc[int(hard_neg.shape[0] * train_frac):, :]


In [43]:
loans_train_realdist.shape, loans_test_realdist.shape,\
equiv_train_realdist.shape, equiv_test_realdist.shape,\
random_train_realdist.shape, random_test_realdist.shape,\
hard_neg_train_realdist.shape, hard_neg_test_realdist.shape, \
loans_train_balanced.shape, loans_test_balanced.shape,\
equiv_train_balanced.shape, equiv_test_balanced.shape,\
random_train_balanced.shape, random_test_balanced.shape,\
hard_neg_train_balanced.shape, hard_neg_test_balanced.shape, \
loans_train_alldata.shape, loans_test_alldata.shape,\
equiv_train_alldata.shape, equiv_test_alldata.shape,\
random_train_alldata.shape, random_test_alldata.shape,\
hard_neg_train_alldata.shape, hard_neg_test_alldata.shape

((1209, 14),
 (135, 14),
 (1612, 14),
 (180, 14),
 (806, 14),
 (90, 14),
 (403, 14),
 (45, 14),
 (1209, 14),
 (135, 14),
 (691, 14),
 (77, 14),
 (345, 14),
 (39, 14),
 (172, 14),
 (20, 14),
 (1209, 14),
 (135, 14),
 (5586, 14),
 (621, 14),
 (1209, 14),
 (135, 14),
 (3853, 14),
 (429, 14))

In [10]:
#random.loc[random['loan_word']=='#NAME?']
# equiv_train.loc[equiv_train['loan_word']=='#NAME?']
# hard_neg_test.loc[hard_neg_test['loan_word']=='#NAME?'] 
# loans.loc[loans['loan_word']=='#NAME?']



In [11]:
# DON't DO THIS FOR NEW Datasets , drop the wiktionary IPA since we are using Epitran IPAs for all sets

# random_train = random_train.drop(["original_word_ipas", "loan_word_ipas"], axis=1)
# random_test = random_test.drop(["original_word_ipas", "loan_word_ipas"], axis=1)
# hard_neg_train = hard_neg_train.drop(["original_word_ipas", "loan_word_ipas"], axis=1)
# hard_neg_test = hard_neg_test.drop(["original_word_ipas", "loan_word_ipas"], axis=1)


In [44]:
#put binary labels for loan and non loan words

loans_train_realdist['label_bin'] = 1
loans_test_realdist['label_bin'] = 1
equiv_train_realdist['label_bin'] = 0
equiv_test_realdist['label_bin'] = 0
random_train_realdist['label_bin'] = 0
random_test_realdist['label_bin'] = 0
hard_neg_train_realdist['label_bin'] = 0
hard_neg_test_realdist['label_bin'] = 0

loans_train_balanced['label_bin'] = 1
loans_test_balanced['label_bin'] = 1
equiv_train_balanced['label_bin'] = 0
equiv_test_balanced['label_bin'] = 0
random_train_balanced['label_bin'] = 0
random_test_balanced['label_bin'] = 0
hard_neg_train_balanced['label_bin'] = 0
hard_neg_test_balanced['label_bin'] = 0

loans_train_alldata['label_bin'] = 1
loans_test_alldata['label_bin'] = 1
equiv_train_alldata['label_bin'] = 0
equiv_test_alldata['label_bin'] = 0
random_train_alldata['label_bin'] = 0
random_test_alldata['label_bin'] = 0
hard_neg_train_alldata['label_bin'] = 0
hard_neg_test_alldata['label_bin'] = 0


/var/folders/xr/f9q8dphs5vq5mhqy843mnkv40000gn/T/ipykernel_9065/1150974597.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans_train_realdist['label_bin'] = 1
/var/folders/xr/f9q8dphs5vq5mhqy843mnkv40000gn/T/ipykernel_9065/1150974597.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  loans_test_realdist['label_bin'] = 1
/var/folders/xr/f9q8dphs5vq5mhqy843mnkv40000gn/T/ipykernel_9065/1150974597.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 

In [45]:
loans_train_realdist.shape, loans_test_realdist.shape,\
equiv_train_realdist.shape, equiv_test_realdist.shape,\
random_train_realdist.shape, random_test_realdist.shape,\
hard_neg_train_realdist.shape, hard_neg_test_realdist.shape, \
loans_train_balanced.shape, loans_test_balanced.shape,\
equiv_train_balanced.shape, equiv_test_balanced.shape,\
random_train_balanced.shape, random_test_balanced.shape,\
hard_neg_train_balanced.shape, hard_neg_test_balanced.shape, \
loans_train_alldata.shape, loans_test_alldata.shape,\
equiv_train_alldata.shape, equiv_test_alldata.shape,\
random_train_alldata.shape, random_test_alldata.shape,\
hard_neg_train_alldata.shape, hard_neg_test_alldata.shape

((1209, 15),
 (135, 15),
 (1612, 15),
 (180, 15),
 (806, 15),
 (90, 15),
 (403, 15),
 (45, 15),
 (1209, 15),
 (135, 15),
 (691, 15),
 (77, 15),
 (345, 15),
 (39, 15),
 (172, 15),
 (20, 15),
 (1209, 15),
 (135, 15),
 (5586, 15),
 (621, 15),
 (1209, 15),
 (135, 15),
 (3853, 15),
 (429, 15))

In [48]:
train_final_realdist = pd.concat([loans_train_realdist, equiv_train_realdist, random_train_realdist, hard_neg_train_realdist])
test_final_realdist = pd.concat([loans_test_realdist, equiv_test_realdist, random_test_realdist, hard_neg_test_realdist])

train_final_balanced = pd.concat([loans_train_balanced, equiv_train_balanced, random_train_balanced, hard_neg_train_balanced])
test_final_balanced = pd.concat([loans_test_balanced, equiv_test_balanced, random_test_balanced, hard_neg_test_balanced])

train_final_alldata = pd.concat([loans_train_alldata, equiv_train_alldata, random_train_alldata, hard_neg_train_alldata])
test_final_alldata = pd.concat([loans_test_alldata, equiv_test_alldata, random_test_alldata, hard_neg_test_alldata])


In [49]:
#shuffle it after final concatenation 

train_final_realdist = train_final_realdist.iloc[np.random.permutation(len(train_final_realdist))]
test_final_realdist = test_final_realdist.iloc[np.random.permutation(len(test_final_realdist))]

train_final_balanced = train_final_balanced.iloc[np.random.permutation(len(train_final_balanced))]
test_final_balanced = test_final_balanced.iloc[np.random.permutation(len(test_final_balanced))]

train_final_alldata = train_final_alldata.iloc[np.random.permutation(len(train_final_alldata))]
test_final_alldata = test_final_alldata.iloc[np.random.permutation(len(test_final_alldata))]

In [51]:
train_final_realdist.shape, test_final_realdist.shape,\
train_final_balanced.shape, test_final_balanced.shape,\
train_final_alldata.shape, test_final_alldata.shape

((4030, 15), (450, 15), (2417, 15), (271, 15), (11857, 15), (1320, 15))

In [52]:
train_final_realdist.to_csv('../Datasets/train_final_production_realdist.csv')
test_final_realdist.to_csv('../Datasets/test_final_production_realdist.csv')

train_final_balanced.to_csv('../Datasets/train_final_production_balanced.csv')
test_final_balanced.to_csv('../Datasets/test_final_production_balanced.csv')

train_final_alldata.to_csv('../Datasets/train_final_production_alldata.csv')
test_final_alldata.to_csv('../Datasets/test_final_production_alldata.csv')

In [53]:
test_final_alldata

,loan_word,original_word,loan_word_epitran,original_word_epitran,loan_english,original_english,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,label,label_bin
2180,नौकर,ناهار,nɔːkər,nɒhɒr,Servant,Lunch,0.666667,0.200000,0.083333,0.083333,1.300000,0.083333,5,hard_negative,0
2185,प्रभाव,نتیجه,prəb̤aːv,ntjd͡ʒh,Effect,Result,1.000000,1.000000,0.319444,0.368056,4.083333,0.336806,6,synonym,0
3050,माही,ماب,maːɦiː,mɒb,lover,map,0.833333,0.500000,0.296875,0.322917,3.187500,0.322917,4,hard_negative,0
818,बरामद,رهائی,bəraːməd,rhɒjʔj,found,رهائی,0.875000,0.714286,0.315476,0.369048,4.607143,0.339286,5,random,0
2571,बराबर,بارآور,bəraːbər,bɒrɒvr,Equal,Fertile,0.625000,0.142857,0.172619,0.190476,1.589286,0.187500,6,hard_negative,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3170,वाहवाही,تشویق و تمجید,vaːɦvaːɦiː,tʃvjɣ v tmd͡ʒjd,Praise,Applause,0.866667,0.727273,0.422348,0.465909,4.568182,0.452652,13,synonym,0
4560,नुकसान पहुचने वाला,خطرناک,nuksaːnə pəɦut͡ʃne vaːlaː,xtrnɒk,Harmful,Dangerous,0.920000,0.722222,0.636574,0.708333,5.333333,0.702546,18,synonym,0
140,कबाब,کباب,kəbaːb,kbɒb,Kebab,Kebab,0.500000,0.200000,0.204167,0.225000,1.750000,0.225000,4,loan,1
4493,अटल,یک دنده,aʈəl,jk dndh,Firm,stubborn,1.000000,0.666667,0.413194,0.465278,4.583333,0.447917,7,synonym,0


In [55]:
equiv.sample(n = 20, random_state = 3)

,loan_word,original_word,loan_word_epitran,original_word_epitran,loan_english,original_english,Fast Levenshtein Distance Div Maxlen,Dolgo Prime Distance Div Maxlen,Feature Edit Distance Div Maxlen,Hamming Feature Distance Div Maxlen,Weighted Feature Distance Div Maxlen,Partial Hamming Feature Distance Div Maxlen,plain Levenshtein,label
913,सक्रिय,مراقب باشید,səkrij,mrɒɣb bɒʃjd,Active,be careful,0.909091,0.600000,0.443750,0.500000,4.187500,0.481250,11,synonym
5356,कुछ,بلند بلند,kut͡ʃʰ,blnd blnd,something,Long long,1.000000,1.000000,0.666667,0.723958,5.968750,0.718750,9,synonym
551,"परेशानी, क्लेश",پریشانی,"pəreʃaːniː, kleʃ",prjʃɒnj,"trouble, trouble",Distress,0.750000,0.500000,0.407986,0.454861,3.625000,0.447917,14,synonym
5782,संक्षिप्त,موقت,səŋkʂiptə,mvɣt,brief,Temporary,0.888889,0.777778,0.564815,0.629630,4.833333,0.620370,9,synonym
4182,अभी,بیچاره,ab̤iː,bjt͡ʃɒrh,Now,the poor,1.000000,0.833333,0.545139,0.618056,5.041667,0.607639,6,synonym
3204,अनुमति न दें,مانع,anumti nə den,mɒnʔ,Do not allow,Barrier,0.846154,0.727273,0.611742,0.674242,5.147727,0.670455,12,synonym
3067,भंगुर,سست,b̤əŋɡur,sst,brittle,Weak,1.000000,1.000000,0.565972,0.638889,5.333333,0.618056,5,synonym
747,सहमत,مهربان,səɦmət,mhrbɒn,Agree,kind,1.000000,0.666667,0.166667,0.194444,3.104167,0.166667,6,synonym
1248,चूक,عیب,t͡ʃuːk,ʔjb,omission,Defect,1.000000,1.000000,0.229167,0.250000,4.000000,0.229167,3,synonym
1739,अधिग्रहण करना,دستگیر کردن,ad̤iɡrəɦɳə kərnaː,dstɡjr krdn,to acquire,arrest,0.705882,0.571429,0.361607,0.416667,4.285714,0.394345,12,synonym


# Discard below?

In [2]:
equiv_words = pd.read_csv("Equivalent.csv")

In [3]:
equiv_words

,loan_word,original_word,loan_word_epitran,original_word_epitran,Fast Levenshtein,Dolgo Prime Distance,Feature Edit Distance,Hamming Feature Distance,Weighted Feature Distance,Fast Levenshtein Distance Div Maxlen
0,छोड़ देना,دست برداشتن از,t͡ʃʰoɽə denaː,dst brdɒʃtn ɒz,0.928571,0.500000,0.352679,0.383929,3.750000,0.928571
1,छोड़ देना,رد کردن,t͡ʃʰoɽə denaː,rd krdn,0.846154,0.461538,0.232372,0.275641,3.048077,0.846154
2,छोड़ देना,دست بردارید,t͡ʃʰoɽə denaː,dst brdɒrjd,0.923077,0.615385,0.258013,0.301282,3.317308,0.923077
3,छोड़ देना,رها کردن,t͡ʃʰoɽə denaː,rhɒ krdn,0.846154,0.461538,0.190705,0.230769,2.615385,0.846154
4,छोड़ देना,کویر,t͡ʃʰoɽə denaː,kvjr,1.000000,0.461538,0.323718,0.375000,3.000000,1.000000
...,...,...,...,...,...,...,...,...,...,...
8445,उमंग,اشتیاق,uməŋɡə,ɒʃtjɒɣ,1.000000,0.833333,0.250000,0.277778,5.270833,1.000000
8446,परोक्ष,مورب,pərokʂə,mvrb,0.857143,0.857143,0.473214,0.523810,4.892857,0.857143
8447,परोक्ष,خودسر,pərokʂə,xvdsr,1.000000,0.857143,0.392857,0.464286,4.375000,1.000000
8448,स्वच्छंद,مورب,svət͡ʃt͡ʃʰəndə,mvrb,0.928571,0.571429,0.361607,0.404762,3.196429,0.928571


In [4]:
hardneg_words = pd.read_csv("Epi_Hard_Neg.csv")

In [5]:
hardneg_words

,loan_word,original_word,loan_word_epitran,original_word_epitran,label,Fast Levenshtein,Dolgo Prime Distance,Feature Edit Distance,Hamming Feature Distance,Weighted Feature Distance,Fast Levenshtein Distance Div Maxlen
0,बंदोबस्त,آبدست,aːbdəstə,ɒbdst,0,0.500000,0.250000,0.239583,0.265625,2.000000,0.500000
1,साज़िश,سازش,saːziʃ,sɒzʃ,1,0.500000,0.166667,0.170139,0.187500,1.458333,0.500000
2,शिकार,دیدار,diːdaːr,djdɒr,0,0.571429,0.142857,0.035714,0.041667,0.625000,0.571429
3,अदा,کدو,kədduː,kdv,0,0.666667,0.500000,0.326389,0.368056,2.791667,0.666667
4,देर,دیر,der,djr,1,0.333333,0.333333,0.041667,0.055556,0.958333,0.333333
...,...,...,...,...,...,...,...,...,...,...,...
12079,शर्बत,شربت,ʃərbət,ʃrbt,1,0.333333,0.333333,0.298611,0.333333,2.416667,0.333333
12080,नवीद,خود,xud,xvd,0,0.333333,0.333333,0.104167,0.125000,2.666667,0.333333
12081,बेवफ़ा,بی‌وفا,bevfaː,bj‌vfɒ,1,0.666667,0.166667,0.041667,0.048611,0.729167,0.666667
12082,आब,او,o,ɒv,0,1.000000,0.500000,0.479167,0.520833,3.875000,1.000000


In [7]:
sum(hardneg_words['label']==0)

10740

In [10]:
%store -r Epi_result

In [11]:
Epi_result

,loan_word,original_word,loan_word_epitran,original_word_epitran,fast levenshtein,Dolgo prime distance,Feature edit distance,Hamming feature distance,Weighted feature distance,Fast Levenshtein distance div maxlen,label,features_loan
0,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
1,ख़राश,طرفدار,xəraːʃ,trfdɒr,5,6,1.916667,2.333333,16.5625,1.000000,0,"[-1, -1, 1, 1, -1, -1, -1, 0, -1, -1, -1, -1, ..."
2,चिराग़,نقاب,t͡ʃiraːɣ,nɣɒb,4,8,1.625000,1.833333,13.5625,1.000000,0,"[-1, -1, 1, -1, 1, -1, -1, 0, -1, -1, -1, -1, ..."
3,मरहम,شوهر,mərɦəm,ʃvhr,4,6,2.625000,3.083333,21.2500,1.000000,0,"[-1, 1, 1, -1, -1, -1, 1, 0, 1, -1, -1, 1, -1,..."
4,हैवानियत,خُوشنَوِیسی,ɦæːvaːnijət,xُvʃnowejsj,5,10,2.833333,3.125000,25.7500,0.909091,0,"[-1, 1, 1, 1, -1, -1, -1, 0, 1, -1, -1, -1, -1..."
...,...,...,...,...,...,...,...,...,...,...,...,...
14779,तहसील,تحصیل,təɦsiːl,thsjl,2,4,1.062500,1.208333,8.8125,0.571429,1,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
14780,तीर,برفی,tiːr,brfj,3,4,1.562500,1.916667,13.2500,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
14781,सितारा,قید,sitaːraː,ɣjd,5,8,2.979167,3.416667,24.2500,1.000000,0,"[-1, -1, 1, 1, -1, -1, -1, 0, -1, -1, -1, 1, 1..."
14782,सबूत,کبابه چینی,səbuːt,kbɒbh t͡ʃjnj,7,10,4.291667,4.750000,34.3750,0.833333,0,"[-1, -1, 1, 1, -1, -1, -1, 0, -1, -1, -1, 1, 1..."


In [14]:
Epi_result.loc[Epi_result['loan_word']=='तसल्ली']

,loan_word,original_word,loan_word_epitran,original_word_epitran,fast levenshtein,Dolgo prime distance,Feature edit distance,Hamming feature distance,Weighted feature distance,Fast Levenshtein distance div maxlen,label,features_loan
0,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
2066,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
2760,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
5301,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
8151,तसल्ली,تسلی,təslliː,tslj,2,4,1.937500,2.166667,15.9375,0.571429,1,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
8276,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
11586,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
11624,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
12952,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
13366,तसल्ली,بغاوت,təslliː,bɣɒvt,6,7,2.020833,2.291667,19.1250,1.000000,0,"[-1, -1, 1, -1, -1, -1, -1, 0, -1, -1, -1, 1, ..."
